We want to plot the power and gaz network that we consider

In [2]:
import pandas as pd
import numpy as np
import folium
import os
from folium.plugins import MarkerCluster

In [3]:
node_nb = 6

DATA_path = os.path.dirname(os.getcwd())+"/DATA"
Pow_dir = DATA_path+'/Power_System_Data'
Gas_dir = DATA_path+'/NG_System_Data'

NG_nodes = pd.read_csv(Gas_dir+'/NG_Nodes.csv')
NG_pipelines = pd.read_csv(Gas_dir+'/NG2NG_Pipelines.csv')
NG_pipelines = NG_pipelines.loc[NG_pipelines['is_existing'] == 1]

NG_2_power = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes/NG_AdjE_Nodes.csv")

Power_nodes = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes"+'/Power_Nodes.csv')
Power_lines = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes"+'/Transmission_Lines.csv')
Power_lines = Power_lines.loc[Power_lines['is_existing'] == 1]

In [4]:
world_map = folium.Map(location=(43, -70), zoom_start=7, tiles="cartodbpositron")

# NG network
for _, row in NG_nodes.iterrows():
    lat, long = row['Lat'], -row['Lon']
    folium.CircleMarker(location=(lat, long), radius=5, color='blue').add_to(world_map)

for _, row in NG_pipelines.iterrows():
    fr_node = NG_nodes.loc[NG_nodes['node_num'] == row['from_node']]
    to_node = NG_nodes.loc[NG_nodes['node_num'] == row['to_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = (to_node['Lat'].values[0], -to_node['Lon'].values[0])
    folium.PolyLine(locations=[fr_node_coords, to_node_coords], color='blue').add_to(world_map)

# Power network
count = 0
for i, row in Power_nodes.iterrows():
    count+=1
    lat, long = row['Lat'], -row['Lon']
    folium.CircleMarker(location=(lat, long), radius=5,  color='red', popup=i).add_to(world_map)

print("There are ", count, " power nodes")
for _, row in Power_lines.iterrows():
    fr_node = Power_nodes.loc[Power_nodes['node_num'] == row['from_node']]
    to_node = Power_nodes.loc[Power_nodes['node_num'] == row['to_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = (to_node['Lat'].values[0], -to_node['Lon'].values[0])
    folium.PolyLine(locations=[fr_node_coords, to_node_coords],color='red').add_to(world_map)


# NG to Power
for _, row in NG_2_power.iterrows():
    fr_node = NG_nodes.loc[NG_nodes['node_num'] == row['NG_node']]
    if fr_node.empty:
        continue
    to_node = []
    for col in NG_2_power.columns[1:]:
        if pd.isna(row[col]):
            continue
        to_node.append(Power_nodes.loc[Power_nodes['node_num'] == row[col]])
    #to_node = Power_nodes.loc[Power_nodes['node_num'] == row['Power_node']]
    fr_node_coords = (fr_node['Lat'].values[0], -fr_node['Lon'].values[0])
    to_node_coords = [(tn['Lat'].values[0], -tn['Lon'].values[0]) for tn in to_node]

    for tn_coords in to_node_coords:
        folium.PolyLine(locations=[fr_node_coords, tn_coords],  color='green').add_to(world_map)
    


legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 120px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;">
    <b>Legend</b><br>
    <span style="display:inline-block; width:12px; height:12px; border: 3px solid blue; border-radius:50%; margin-right:5px;"></span> NG Node<br>
    <span style="display:inline-block; width:12px; height:12px; border: 3px solid red; border-radius:50%; margin-right:5px;"></span> Power Node<br>
    <span style="display:inline-block; width:20px; height:3px; background-color:blue; margin-right:5px;"></span> NG Pipeline<br>
    <span style="display:inline-block; width:20px; height:3px; background-color:red; margin-right:5px;"></span> Power Line<br>
    <span style="display:inline-block; width:20px; height:2px; background-color:green; margin-right:5px;"></span> NG to Power<br>
</div>
'''

world_map.get_root().html.add_child(folium.Element(legend_html))
world_map

There are  6  power nodes


### Assignement of each power node to the closest NG node

In [18]:
from geopy.distance import geodesic

# Function to calculate the distance between two points
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Create a new dataframe to store the closest NG node for each power node
#closest_ng_nodes = pd.DataFrame(columns=['Power_node', 'Closest_NG_node'])

node_nb = 17

DATA_path = os.path.dirname(os.getcwd())+"/DATA"
Pow_dir = DATA_path+'/Power_System_Data'
Gas_dir = DATA_path+'/NG_System_Data'

NG_nodes = pd.read_csv(Gas_dir+'/NG_Nodes.csv')
Power_nodes = pd.read_csv(Pow_dir+f"/{node_nb}-Nodes"+'/Power_Nodes.csv')

NG = [(row['Lat'], -row['Lon']) for _, row in NG_nodes.iterrows()]

#closest_Power_node = np.zeros(len(NG))
closest_ng_node = {i : [] for i in range(len(NG))}

# Iterate through each power node
for i, power_node in Power_nodes.iterrows():
    power_node_coords = (power_node['Lat'], -power_node['Lon'])

    dist = [calculate_distance(power_node_coords, ng) for ng in NG]
    closest_ng = np.argmin(dist)

    closest_ng_node[closest_ng].append(i)

l_max = np.max([len(v) for v in closest_ng_node.values()])
for k in closest_ng_node.keys():
    l = len(closest_ng_node[k])
    closest_ng_node[k] += [np.nan] * (l_max - len(closest_ng_node[k]))

ng_n = np.array([k for k in closest_ng_node.keys()]).reshape(-1, 1)
print(ng_n)
pw_n = np.array([v for v in closest_ng_node.values()])
col = ['NG_node']+[f'Power_node_{i}' for i in range(l_max)]

df = pd.DataFrame(np.concatenate([ng_n, pw_n], axis = 1), columns=col )
#df.to_csv('NG_AdjE_Nodes.csv', index=False)
df

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]]


,NG_node,Power_node_0,Power_node_1,Power_node_2,Power_node_3
0,0.0,13.0,NaN,NaN,NaN
1,1.0,15.0,NaN,NaN,NaN
2,2.0,14.0,NaN,NaN,NaN
3,3.0,16.0,NaN,NaN,NaN
4,4.0,7.0,NaN,NaN,NaN
5,5.0,NaN,NaN,NaN,NaN
6,6.0,8.0,11.0,NaN,NaN
7,7.0,NaN,NaN,NaN,NaN
8,8.0,0.0,NaN,NaN,NaN
9,9.0,6.0,NaN,NaN,NaN


In [19]:
d = {1:[4]}
d[1]+=[5,6]
d

{1: [4, 5, 6]}